In [2]:
#Conexion a la base de datos AdventureWorks en SQL Server database
import pyodbc
import pandas as pd
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"  # O el driver que tengas instalado
    "SERVER=ADRIANAC\SQLEXPRESS;" #Nombre o direccion IP del SQL server
    "DATABASE=AdventureWorks2022;" #Nombre de la base de datos que tenog en SQL y quiero usar
    "Trusted_Connection=yes;" #Porque tengo autenticacion de windows, sino, me pediria usuario y psw
)

# Establecer la conexión
with pyodbc.connect(connection_string) as conn:
    # Crear el cursor
    cursor = conn.cursor()

<>:6: SyntaxWarning: invalid escape sequence '\S'
<>:6: SyntaxWarning: invalid escape sequence '\S'
C:\Users\adria\AppData\Local\Temp\ipykernel_3372\881769128.py:6: SyntaxWarning: invalid escape sequence '\S'
  "SERVER=ADRIANAC\SQLEXPRESS;" #Nombre o direccion IP del SQL server


In [14]:
#Realizar consultas SQL en python
try:
    # Establecer la conexión
    connection = pyodbc.connect(connection_string)

    # Consulta SQL para leer la tabla
    query = "SELECT * FROM Person.Person"

    # Leer datos en un DataFrame de Pandas
    df = pd.read_sql(query, connection)
   
    # Mostrar las primeras filas
    print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

except pyodbc.Error as ex:
    sqlstate = ex.args[1]
    print(f"Error al conectar o ejecutar la consulta: {sqlstate}")




C:\Users\adria\AppData\Local\Temp\ipykernel_12100\2035169001.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


| BusinessEntityID   | PersonType   | NameStyle   | Title   | FirstName   | MiddleName   | LastName   | Suffix   | EmailPromotion   | AdditionalContactInfo   | Demographics                                                                                                                                                        | rowguid                              | ModifiedDate        |
|:-------------------|:-------------|:------------|:--------|:------------|:-------------|:-----------|:---------|:-----------------|:------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------|:--------------------|
| 1                  | EM           | False       |         | Ken         | J            | Sánchez    |          | 0                |                         | <IndividualSurvey xmlns="http://schemas.microsoft.com/sqlserver/20

In [3]:
#Filtra los empleados de la tabla Person.Person que viven en la ciudad de 'Seattle'.

query = f"""
SELECT pp.FirstName AS Nombre, pp.LastName AS Apellido, city AS Ciudad
FROM Person.Person pp 
LEFT JOIN Person.BusinessEntityAddress bea ON pp.BusinessEntityID = bea.BusinessEntityID 
LEFT JOIN Person.Address pa ON bea.AddressID = pa.AddressID 
WHERE City='Seattle'
"""
# Leer datos en un DataFrame de Pandas
df = pd.read_sql(query, connection)

# Mostrar las primeras filas
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))



| Nombre   | Apellido   | Ciudad   |
|:---------|:-----------|:---------|
| Pilar    | Ackerman   | Seattle  |
| Elijah   | Adams      | Seattle  |
| Jason    | Adams      | Seattle  |
| Jack     | Adams      | Seattle  |
| Robert   | Alexander  | Seattle  |


C:\Users\adria\AppData\Local\Temp\ipykernel_12100\2691514341.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [5]:
#Calcula el total de ventas por año a partir de la tabla Sales.SalesOrderHeader.

query1=f"""
SELECT YEAR(soh.OrderDate) AS año, FORMAT(ROUND(SUM(sod.OrderQty * sod.UnitPrice), 0), 'N0') AS totalVentas
FROM Sales.SalesOrderDetail sod
LEFT JOIN Sales.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
GROUP BY YEAR (soh.OrderDate)
ORDER BY YEAR (soh.OrderDate) ASC
"""

# Leer datos en un DataFrame de Pandas
df = pd.read_sql(query1, connection)

# Mostrar las primeras filas
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

C:\Users\adria\AppData\Local\Temp\ipykernel_12100\2774882196.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query1, connection)


| año   | totalVentas   |
|:------|:--------------|
| 2011  | 12,646,112    |
| 2012  | 33,710,897    |
| 2013  | 43,922,051    |
| 2014  | 20,094,830    |


In [7]:
#Agrupa los productos de la tabla Production.Product por categoría y cuenta cuántos productos hay en cada categoría.
query=f"""
SELECT pc.Name AS NombreCategoria, COUNT(pp.ProductID) AS Cantidad
FROM Production.Product pp 
INNER JOIN Production.ProductSubcategory ps ON pp.ProductSubcategoryID = ps.ProductSubcategoryID
AND ps.ProductCategoryID IS NOT NULL
INNER JOIN Production.ProductCategory pc ON ps.ProductCategoryID = pc.ProductCategoryID
GROUP BY pc.Name,(pc.ProductCategoryID)
"""
# Leer datos en un DataFrame de Pandas
df = pd.read_sql(query, connection)

# Mostrar las primeras filas
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Guardar el DataFrame en un archivo CSV
df.to_csv('resultados_productos.csv', index=False)  # index=False evita guardar el índice de las filas

| NombreCategoria   | Cantidad   |
|:------------------|:-----------|
| Accessories       | 29         |
| Bikes             | 97         |
| Clothing          | 35         |
| Components        | 134        |


C:\Users\adria\AppData\Local\Temp\ipykernel_12100\1803914574.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [9]:
# Eliminar la tabla existente si existe
cursor.execute("DROP TABLE IF EXISTS Silver_EmpleadosSeattle;")
conn.commit()# asegura que todas las operaciones dentro de la transacción se completen exitosamente 
#y que los cambios se guarden en la base de datos. Si no llamas a conn.commit(), 
#los cambios no se reflejarán en la base de datos y se descartarán cuando se cierre la conexión.

# Crear la tabla EmpleadosSeattle con las columnas de la consulta
create_table_query = """
CREATE TABLE Silver_EmpleadosSeattle(
    BusinessEntityID INT PRIMARY KEY,
    FirstName NVARCHAR(50),
    LastName NVARCHAR(50),
    City NVARCHAR(50)
);
"""
cursor.execute(create_table_query)
conn.commit()

# Insertar empleados de Seattle
# Usamos el DISTINCT para asegurarnos de que no haya filas duplicadas en el resultado de la consulta que se insertará en la tabla EmpleadosSeattle
insert_query = """
INSERT INTO Silver_EmpleadosSeattle (BusinessEntityID, FirstName, LastName, City)
SELECT DISTINCT pp.BusinessEntityID, pp.FirstName, pp.LastName, pa.City #
FROM Person.Person pp
INNER JOIN Person.BusinessEntityAddress bea ON pp.BusinessEntityID = bea.BusinessEntityID
INNER JOIN Person.Address pa ON bea.AddressID = pa.AddressID
WHERE pa.City = 'Seattle';
"""
cursor.execute(insert_query)
conn.commit()

# Leer datos en un DataFrame de Pandas
query = "SELECT * FROM Silver_EmpleadosSeattle;"
df = pd.read_sql(query, conn)

# Mostrar las primeras filas
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

| BusinessEntityID   | FirstName   | LastName    | City    |
|:-------------------|:------------|:------------|:--------|
| 33                 | Annik       | Stahl       | Seattle |
| 36                 | Jose        | Lugo        | Seattle |
| 39                 | Ed          | Dudenhoefer | Seattle |
| 40                 | JoLynn      | Dobney      | Seattle |
| 58                 | Kendall     | Keil        | Seattle |


C:\Users\adria\AppData\Local\Temp\ipykernel_3372\224792119.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
